In [3]:
import os
import numpy as np
import pandas as pd
import requests
import seaborn as sns
import plotly.express as px

In [4]:
# Load API-Key
API_KEY = os.getenv('ALPHAVANTAGE_API_KEY')

In [5]:
my_stocks = pd.DataFrame({'Date':[],'Symbol':[],'Price':[],'Perc_Change_From_Initial':[]})
my_stocks_initial_value = pd.DataFrame(columns=['Symbol','Initial Date', 'Initial Price'])

In [6]:

# symbols = ['PRNHX', 'MFEJX', 'VFIAX']

# for symbol in symbols:


#     # Get closing trading data and add as column to my_stocks
#     url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=full&apikey={ALPHAVANTAGE_API_KEY}'
#     response = requests.get(url)
#     symbol_data = response.json()

#     trading_data = symbol_data['Time Series (Daily)']

#     close_data_series = pd.to_numeric(pd.DataFrame(trading_data).T['4. close'].rename(symbol))

#     my_stocks[symbol] = close_data_series


#     # Get minimum date per stock and append to my_stocks_initial_value
#     initial_date = close_data_series.index.min()
#     initial_price = close_data_series[initial_date]

#     my_stocks_initial_value = my_stocks_initial_value.append({'Symbol':symbol, 'Initial Date':initial_date, 'Initial Price':initial_price}, ignore_index=True)


#     # Create and add series of evolution of price percentage increase as column to my_stocks
#     my_stocks[f'{symbol}_percentage_change'] = ((close_data_series / initial_price) - 1) * 100

# my_stocks = my_stocks.reset_index().rename(columns={'index':'Date'})

In [8]:
symbols = ['PRNHX', 'MFEJX', 'VFIAX']

for symbol in symbols:

    # Get closing trading data
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=full&apikey={API_KEY}'
    response = requests.get(url)
    symbol_data = response.json()

    trading_data = symbol_data['Time Series (Daily)']

    close_data_series = pd.to_numeric(pd.DataFrame(trading_data).T['4. close'].rename(symbol))


    # Get minimum date per stock and append to my_stocks_initial_value
    initial_date = close_data_series.index.min()
    initial_price = close_data_series[initial_date]

    my_stocks_initial_value = my_stocks_initial_value.append({'Symbol':symbol, 'Initial Date':initial_date, 'Initial Price':initial_price}, ignore_index=True)


    # Create and add series of evolution of price percentage increase as column to my_stocks
    this_symbol_perc_chg = ((close_data_series / initial_price) - 1) * 100


    # Bring together data
    this_stock_df = pd.DataFrame(close_data_series).reset_index().rename(columns={'index':'Date',f'{symbol}':'Price'})
    this_stock_df['Symbol'] = symbol
    this_stock_df['Perc_Change_From_Initial'] = this_symbol_perc_chg.array
    this_stock_df = this_stock_df[['Date','Symbol','Price','Perc_Change_From_Initial']]

    # Append data to my_stocks
    my_stocks = my_stocks.append(this_stock_df)

In [9]:
my_stocks

,Date,Symbol,Price,Perc_Change_From_Initial
0,2022-05-20,PRNHX,47.78,93.363011
1,2022-05-19,PRNHX,47.70,93.039255
2,2022-05-18,PRNHX,46.36,87.616350
3,2022-05-17,PRNHX,48.69,97.045730
4,2022-05-16,PRNHX,47.89,93.808175
...,...,...,...,...
5409,2000-11-17,VFIAX,126.44,1.249199
5410,2000-11-16,VFIAX,126.86,1.585522
5411,2000-11-15,VFIAX,128.47,2.874760
5412,2000-11-14,VFIAX,127.82,2.354260


In [10]:
my_stocks_initial_value

,Symbol,Initial Date,Initial Price
0,PRNHX,1999-11-01,24.71
1,MFEJX,2005-03-31,30.11
2,VFIAX,2000-11-13,124.88


In [11]:
# Save results to .csv's
my_stocks.to_csv(r'C:\Users\Noah\Desktop\API-Cloud-Stocks\my_stocks')
my_stocks_initial_value.to_csv(r'C:\Users\Noah\Desktop\API-Cloud-Stocks\my_stocks_initial_Value')